# ADM - HW 3

# 1. Data Collection

## 1.1 Get the list of master's degree courses

In [ ]:
# se to true to install bs4 & selenium
if False: 
    pip install bs4
    pip install selenium

In [1]:
from bs4 import BeautifulSoup
import requests
import os 
import itertools
import time

First of all, we get a list of all the master's degree courses we need.

In [2]:
def extract_masters(this_url):
  result_url = requests.get(this_url)
  result_soup = BeautifulSoup(result_url.text)
  result_links = result_soup.find_all('a', {'class':'courseLink'})
  result_list=[]
  for item in result_links:
    result_list.append((item['href'], item.text))

  return result_list

We notice that the url of th various pages is *https://www.findamasters.com/masters-degrees/msc-degrees/?PG=* followed by the number of the page in the MSc courses catalogue we're in. 

In [3]:
links = []
for page in range(1, 401):
    url = 'https://www.findamasters.com/masters-degrees/msc-degrees/?PG='+str(page)
    links += extract_masters(url)
    print(page, len(links))

1 15
2 30
3 45
4 60
5 75
6 90
7 105
8 120
9 135
10 150
11 165
12 180
13 195
14 210
15 225
16 240
17 255
18 270
19 285
20 300
21 315
22 330
23 345
24 360
25 375
26 390
27 405
28 420
29 435
30 450
31 465
32 480
33 495
34 510
35 525
36 540
37 555
38 570
39 585
40 600
41 615
42 630
43 645
44 660
45 675
46 690
47 705
48 720
49 735
50 750
51 765
52 780
53 795
54 810
55 825
56 840
57 855
58 870
59 885
60 900
61 915
62 930
63 945
64 960
65 975
66 990
67 1005
68 1020
69 1035
70 1050
71 1065
72 1080
73 1095
74 1110
75 1125
76 1140
77 1155
78 1170
79 1185
80 1200
81 1215
82 1230
83 1245
84 1260
85 1275
86 1290
87 1305
88 1320
89 1335
90 1350
91 1365
92 1380
93 1395
94 1410
95 1425
96 1440
97 1455
98 1470
99 1485
100 1500
101 1515
102 1530
103 1545
104 1560
105 1575
106 1590
107 1605
108 1620
109 1635
110 1650
111 1665
112 1680
113 1695
114 1710
115 1725
116 1740
117 1755
118 1770
119 1785
120 1800
121 1815
122 1830
123 1845
124 1860
125 1875
126 1890
127 1905
128 1920
129 1935
130 1950
131 1965
1

We want to verify we retrieved all of the needed urls, so we check for the length of the list

In [5]:
print(len(links))

6000


It's the expected length, so we can create and write the required *.txt* file.   

In [4]:
file = open('msc_links.txt','w')
for link in links:
	file.write(link[0]+'\n')

file.close()

Now, if we look at the file it's written in the correct way (:

## 1.2 Crawl master's degree pages 

In [62]:
def download_html(url, dest_folder):
    #path is the local path we're working in 
    if not os.path.exists(dest_folder):  # create folder if it does not exist
        os.makedirs(dest_folder) 
    
    filename = url.replace('https://www.findamasters.com//masters-degrees/course/', '').replace('/', '_')
    file_path = os.path.join(dest_folder, filename)
    url_req = requests.get(url)

    if url_req.ok:
        soup = BeautifulSoup(url_req.text, 'html.parser')
        with open(file_path, 'w') as file: 
            file.write(soup.prettify())
        
        file.close()
        

In [16]:
### SAVING HTML & ORGANIZING THEM IN FOLDERS ###
with open('msc_links.txt', 'r') as file:
    for i in range(1, 5):
        folder_i = 'page_' + str(i)
        
        #selecting the lines corresponding to the i-th page
        needed_lines = itertools.islice(file, 0, 15)
        
        #saving the corresponding html in "page_i" folder
        for line in needed_lines:
            url  = 'https://www.findamasters.com/' + str(line)[:-1]
            download_html(str(url), folder_i)
        
        time.sleep(2)

In [68]:
### SAVING HTML & ORGANIZING THEM IN FOLDERS ###
file = open('msc_links.txt', 'r')
lines_list = file.readlines()
file.close()

In [73]:
for i in range(47, 401):
    folder_i = 'page_' + str(i)
        
    #selecting the lines corresponding to the i-th page
    start = (i-1)*15
    stop =  i*15
    needed_lines = lines_list[start : stop]

    #saving the corresponding html in "page_i" folder
    for line in needed_lines:
        url  = 'https://www.findamasters.com/' + str(line)[:-1]
        download_html(str(url), folder_i)
        time.sleep(2)
    

## 1.3 Parse downloaded pages

In [ ]:
def get_attributes(doc):
    page_soup = BeautifulSoup(str(doc))

    #Course Name
    page_courseNamelinks = page_soup.find_all('a', {'class': 'course-header__course-title'})
    courseName = page_courseNamelinks[0].contents[0]
    
    
    #University Name 
    page_universityNamelinks = page_soup.find_all('a', {'class': 'course-header__institution'})
    universityName = page_universityNamelinks[0].contents[0]


    #Faculty Name 
    facultyName_links= page_soup.find_all('a', {'class':'course-header__department' })
    facultyName = facultyName_links[0].contents[0]


    #Full Time
    isItFullTime_modality__links= page_soup.find_all('a', {'class':'concealLink' })
    for item in isItFullTime_modality__links:
        if item['href']== "/masters-degrees/full-time/":
            isItFullTime = 'Full Time'
            break
        elif item['href']== "/masters-degrees/part-time/" :
            isItFullTime = 'Part Time'
            break
    

    #Short Description (description)
    descritpion_ref = page_soup.find_all('div', {'id': 'Snippet'})
    description_toclean = str(descritpion_ref[0]).lstrip('<div id="Snippet">\n<p>\n ').rstrip('</p>\n</div> ').split('<p>')
    for i in range(len(description_toclean)): 
        description_toclean[i] = description_toclean[i].lstrip('</p>\n ').rstrip('</p>\n ')

    description = ' '.join(description_toclean)


    # Start Date (startDate)
    startDaterefs = page_soup.find_all('span', {'class':'key-info__start-date'})
    startDate=startDaterefs[0].contents[-1].strip('\n ')
    

    #Fees (to save as fees): string;
    fees_ref = page_soup.find_all('div', {'class': 'course-sections__fees'})
    fees_ref_ref = fees_ref[0].find_all('div', {'class': 'course-sections__content'} )

    fees_link = fees_ref_ref[0].find_all('a')
    if fees_link != []:
        fees = fees_link[0].contents[0].strip('\n ')
    else:
        fees = str(fees_ref_ref[0].contents[1]).strip('<p> </p>\n ')


    #modality (MSC, modality)
    #isItFullTime_modality__links= page_soup.find_all('a', {'class':'concealLink' })
    for item in isItFullTime_modality__links:
        if item['href']== "/masters-degrees/msc-degrees/":
            modality = item.contents[0].strip('\n ')
            break
 

    #Duration (to save as duration):string;
    durationref = page_soup.find_all('span', {'class':'key-info__duration'})
    duration = durationref[0].contents[-1].strip('\n ')

    #City (to save as city): string;
    city_links= page_soup.find_all('a', {'class':'course-data__city' })
    city  = city_links[0].contents[0].strip('\n ')


    #Country (to save as country): string;
    country_links= page_soup.find_all('a', {'class':'course-data__country' })
    country = country_links[0].contents[0].strip('\n ')


    #Presence or online modality (to save as administration): string;
    administration = 'Online'
    administration_links= page_soup.find_all('a', {'class':'course-data__online' })
    if administration_links == []:
        administration = 'On campus'


    #Link to the page (to save as url): string.
    